__Import Libraries__

In [4]:
import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup

__Collect data using Beautiful soup__

In [36]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
html1 = BeautifulSoup(data.content)

In [44]:
soup = BeautifulSoup(str(html1))

In [45]:
table = soup.find('table')
table_str = str(table.extract())

In [46]:
df = pd.read_html(table_str)[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


__Data Preprocessing__

In [47]:
df.shape

(180, 3)

In [49]:
df["Borough"].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

__Remove Not Assigned Values__

In [50]:
NA_value = df[df["Borough"]=="Not assigned"]
df.drop(NA_value.index,inplace=True)

In [51]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [53]:
df = df.groupby(["Postal Code","Borough"])["Neighborhood"].apply(lambda x : ", ".join(x)).to_frame()

In [54]:
df.reset_index(inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [55]:
df[df["Neighborhood"]=="Not assigned"]

,Postal Code,Borough,Neighborhood


In [56]:
df.shape

(103, 3)

__Collect Geospatial Dataframe__

In [58]:
geo_url = "https://cocl.us/Geospatial_data"

geocode_df = pd.read_csv(geo_url)
geocode_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


__Merge Both Dataframes___

In [59]:
final_df = pd.merge(df,geocode_df,on="Postal Code") 

In [62]:
final_df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
